### Problem Statement:

Make me a nice library of routines for fitting input distributions.
The input to a fitting routine for a particular distribution should be a bunch of
observations; the output should be the maximum likelihood estimate. You can use
your favorite high-level language like C++, Java, Python, Matlab, or even Excel.
Include in your library maximum likelihood estimation routines for fitting all of
the usual discrete and continuous distributions, e.g., Bern(p), Geom(p), Exp(λ),
Normal(μ, σ2), Gamma(α, β), Weibull(α, β), etc., etc. Beware! The Weibull takes
a little work (so does the gamma, for that matter). Luckily, everything is outlined
very clearly in my notes and/or Law (2015). Include in your write-up an easy user’s
guide, complete source code, and some appropriate examples. The examples should
be along the following illustrative lines: Generate some example data; attempt to
fit the data to various distributions; conduct χ2 goodness-of-fit tests to show how
well the fits do. For instance, if you generate some example Weibull data, a Weibull
fit ought to do better than an exponential fit

In [1]:
import numpy as np
from scipy.stats import chisquare
import scipy.stats as stats
import pandas as pd

### Data Generation Function:

Function below generates data for different distribution types including 'normal', 'geometric', 'gamma', 'binomial', 'exponential', and 'weibull'. parameters for this function are as below: <br>
    - dist_type: which is a string format indicating the distribution type we are looking for <br>
    - par: is the required parameters for each distribution type <br>
    - row_count: is the size of the data <br>  
More distribution types can be added to the function later. <br>


In [2]:
# Define a function to generate different data types

row_count=10000

def data_generation(dist_type, par, row_count):
    if dist_type== 'normal':
        mean, std = par
        data=np.random.normal(mean, std, row_count)
    
    elif dist_type== 'geometric':
        p= par
        data=np.random.geometric(p, row_count)  
       
    elif dist_type== 'binomial':
        n, p= par
        data=np.random.binomial(n, p, row_count)
    
    elif dist_type == 'poisson':
        l= params
        data = np.random.poisson(l, row_count)    
    
    elif dist_type== 'exponential':
        scale= par
        data=np.random.exponential(scale, row_count)
   
    elif dist_type == 'gamma':
        shape , scale = par
        data=np.random.gamma(shape, scale, row_count)
    
    elif dist_type == 'weibull':
        a= par
        data=np.random.weibull(a, row_count)   
    
    else:
        raise ValueError("Change distribution type or modify parameters")
        
    return data

In [ ]:
def chi_square_test(observed, expected):
    chi_square_statistic = np.sum((observed - expected) ** 2 / expected)
    df = len(observed) - 1
    p_value = 1 - stats.chi2.cdf(chi_square_statistic, df)
    return chi_square_statistic, p_value

In [58]:
# I start with a normal distribution data:
row_count=10000
data=data_generation('normal', [0,1], row_count)
data

array([-2.56354711, -0.1950184 , -1.4253714 , ...,  0.51562144,
        0.78919768, -0.93561795])

## Fitting Different Types of Distributions on Generated Data

In the function below, I intend to fit various types of distributions to the generated data and then perform goodness-of-fit tests for each distribution type. Additionally, I created a dictionary that stores the parameters of each fit along with their respective chi-squared statistics. The next step will involve comparing these fits to identify the best-fitting distribution by assessing their respective chi-squared statistic values. As of now, I have initiated this process with normal and exponential fits, and I plan to expand this to include more distribution types later.


In [70]:
#fit the data to various distributions
def fit_data_to_distributions(data):
    fitted_distributions = {}

    #fit data to the Normal distribution
    norm_params = stats.norm.fit(data)
    #print(norm_params)
    fitted_distributions['normal'] = norm_params
    actual=np.histogram(data, bins=100)[0]
    #print(actual)
    x=np.linspace(data.min(), data.max(), 100)
    expected_data = stats.norm.pdf(x, *norm_params)
    chi2_test_statistic, p_value = chi_square_test(actual, expected_data)
    #chi2_test_statistic, p_value = stats.chisquare(actual, expected_data) 
    fitted_distributions['chi2_norm']= chi2_test_statistic

    #fit data to the Exponential distribution
    exp_params = stats.expon.fit(data)
    fitted_distributions['exponential'] = exp_params
    expected_data = stats.expon.pdf(x, *exp_params)
    chi2_test_statistic, p_value = chi_square_test(actual, expected_data)
    #chi2_test_statistic, p_value = stats.chisquare(actual, expected_data) 
    fitted_distributions['chi2_exp']= chi2_test_statistic
    
    return fitted_distributions

fitted_distributions = fit_data_to_distributions(data)


In [71]:
fitted_distributions

{'normal': (0.0017798411305455246, 0.9967940519739784),
 'chi2_norm': 8077401.667357394,
 'exponential': (-4.016555114750013, 4.018334955880559),
 'chi2_exp': 24802722.556280952}